In [1]:
import random
import time
import pickle
import sys
import json
import re
import string
import collections
import codecs
from io import BytesIO

import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#import matplotlib as mpl

#from sklearn import feature_extraction

#import gensim
#from gensim import corpora
#from gensim.corpora import WikiCorpus
#from gensim.models import Word2Vec
#from gensim.models.word2vec import LineSentence

#import pyLDAvis
#import pyLDAvis.gensim

import nltk
from nltk.corpus import stopwords
from nltk.tag.sequential import ClassifierBasedTagger
#from nltk.stem.wordnet import WordNetLemmatizer
#from nltk.tokenize import word_tokenize

import pygraphviz
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout


from IPython.display import display, Image
from IPython.core.interactiveshell import InteractiveShell

pd.options.mode.chained_assignment = None  # default='warn'

### Classifying the questions

In [2]:
frage_complete = pd.read_pickle('./frage.pickle')

In [3]:
frage_complete[['id', 'fragebogen_num', 'frage_num', 'original_frage', 'kurzfrage',]].head()

,id,fragebogen_num,frage_num,original_frage,kurzfrage
0,1,1,A1,"Kopf: Kopf, Haupt; auch scherzh./übertr.","Kopf, Haupt; auch scherzh./übertr."
1,2,1,A2,Kopf: Kopf/Haupt (in urspr. Bed.) in Vergl./Ra...,Kopf/Haupt (urspr.Bed.) in Vergl./Ra.*
2,3,1,A3,Kopf: Kopf/Haupt (übertr.) in Vergl./Ra. (das ...,Kopf/Haupt (übertr.) in Vergl./Ra.*
3,4,1,A4,"Kopf: schöner, ebenmäßig gebauter Kopf (Christ...","schöner, ebenmäßiger Kopf (Christus-, Titusk.)"
4,5,1,A4a,"Kopf: häßlicher, unebenmäßiger Kopf","häßlicher, unebenmäßiger Kopf"


In [4]:
frage_complete[['id', 'fragebogen_num', 'frage_num', 'original_frage', 'kurzfrage',]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16738 entries, 0 to 16743
Data columns (total 5 columns):
id                16738 non-null int64
fragebogen_num    16738 non-null int64
frage_num         16738 non-null object
original_frage    16738 non-null object
kurzfrage         16738 non-null object
dtypes: int64(2), object(3)
memory usage: 784.6+ KB


In [5]:
#frage_complete[frage_complete.original_frage.str.contains(pat = '\?')]['original_frage']
frage_complete[frage_complete.original_frage.str.contains(pat = 'wie |wer |was ', regex = True)]['original_frage']

116      Palmso.: Palmbuschen, Sachl.: woraus besteht e...
118      Palmso.: was geschieht m. d. Palmbuschen/-gert...
124      Palmso.: Palmesel; auch als SpottN?; Ra. (aufg...
126      Palmso.: wird bei d. Palmeselfahrt e. Christbi...
129      Palmso.: wie heißt Woche vor d. Palmsonntag? h...
                               ...                        
16466    Gewerbe: Goldarbeiter, Goldschmied; Ra. wie: D...
16472    Gewerbe: Gold; Ra. wie: Es ist nicht alles Gol...
16530    Gewerbe: Drechslerhandwerk: Ra. wie: ich werde...
16556    Gewerbe: Bürstenbinder; Ra. wie: saufen wie ei...
16719    Gewerbe: Lohn, Pl.; Ra. wie: einem seinen Lohn...
Name: original_frage, Length: 928, dtype: object

#### Questions patterns

In [ ]:
onomasiological_naming = ['Bez.', '-bez.', 'Bezeichng.','wie nennt man','was bezeichnet',
                          'wie heißen','wie heißt', 'Benennung', 'Benenng']
onomasiological_drawing = []
semasiological_meaning = ['Bedeutung', 'Bed.', 'was bedeutet']
agreement_yes_no = []
agreement_pronunciation = ['Ltg.','Lautung','Lautungen']
description_events = ['Beschr.','Beschreibg.','Beschreibung']
description_sayings = ['Ausdrücke']
illustration_drawing = []
morphological_compound_1 = ['Komp.']
morphological_compound_2 = ['Gw.']
morphological_compound_3 = ['Bw']
morphological_diminutive = ['Dem.']
morphological_plural = ['Plural','Pl.']
morphological_forms = []
morphological_derivatives = ['Abl.','Ableitung']
syntactic_constructions = ['Fügungen', 'Wendungen','Füg.']
syntactic_idioms = ['Sprichw.', 'Ra.']
phonological_pronunciation = ['Aussprache','Ausspr.', 'Ltg.']
metaphorical_conveyed_meanings = ['Übertr.', 'übertragener Bedeutung']
synonyms = ['Syn.', 'Synonym']
synonyms_contrast = ['Ggs.','Gegensatz'] 
synonyms_comparison = ['Vgl.','Vergl.','Vergleich']
cultural_beliefs = ['Aberglaube','Volksaberglaube','Volksglaube']
cultural_event = ['Bräuche', 'Brauch', 'dabei', 'Ausdrücke']
cultural_medicine = ['Volksheilmittel','Heilmittel','Blutreinigungsmittel','volkstüml','Gegenmittel']
cultural_prayer = ['Gebete']
cultural_songs = ['Lieder','Gesangs']
cultural_humor = ['Volkswitze','Mesnerwitze']
cultural_games_1 = ['Finger','Spiel','Kinderspiel','Laufspiele','spielen']
cultural_games_2 = ['Ausdrücke', 'Spiel']
cultural_food = ['zutat','speise','braten','Gebäck','Apfel','Kartoffel','Essen','Bier','trinken']
cultural_living_organisms = []
cultural_dances = ['Tänze','Tanz']

pattern_list = ['onomasiological_naming', 'onomasiological_drawing',
                'semasiological_meaning', 
                'agreement_yes_no', 'agreement_pronunciation', 
                'description_events', 'description_sayings', 
                'illustration_drawing', 
                'morphological_compound_1', 'morphological_compound_2', 'morphological_compound_3',
                'morphological_diminutive', 'morphological_plural', 'morphological_forms', 'morphological_derivatives', 
                'syntactic_constructions', 'syntactic_idioms',
                'phonological_pronunciation',
                'metaphorical_conveyed_meanings',
                'synonyms', 'synonyms_contrast', 'synonyms_comparison',
                'cultural_beliefs', 'cultural_event', 'cultural_medicine', 'cultural_prayer', 
                'cultural_songs', 'cultural_humor', 'cultural_games_1', 'cultural_games_2', 
                'cultural_food', 'cultural_living_organisms', 'cultural_dances'
               ]

In [ ]:
def test_type(entry):
    question_type = None
    for patterns in pattern_list:
        if eval(patterns):
            for pattern in eval(patterns):
                if pattern.lower() in entry.lower():
                    if question_type == None:
                        question_type = [patterns]
                    elif question_type == [patterns]:
                        break
                    else:
                        question_type.append(patterns)
                    break
    return question_type

In [ ]:
frage_complete.original_frage.apply(test_type).dropna() #.unique() #value_counts(dropna=False)

In [ ]:
frage_complete['frage_type'] = frage_complete.original_frage.apply(test_type)
print(f"{len(frage_complete['frage_type'].dropna())} questions were classified out of {len(frage_complete)}")

In [ ]:
frage_complete.loc[frage_complete.original_frage.apply(test_type).isna(), 'original_frage']